# Clasificador Naive Bayes de noticias argentinas

<a target="_blank" href="https://colab.research.google.com/github/pdomins/bayesian-learning/blob/master/ej2_bayes_news.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

El objetivo de este ejercicio es implementar un clasificador de texto utilizando el **clasificador ingenuo de Bayes** sobre el
conjunto de datos *”Noticias Argentinas”* para clasificar cada noticia según su tipo.

Instalaciones previas necesarias:

In [1]:
%%capture
!python -m pip install nltk       # for stopwords
!python -m pip install spacy      # for tokenization
!python -m pip install openpyxl   # for file reading
!python -m spacy download es_core_news_md   # for lemmatization
!python -m pip install scikit-learn         # for integrals

Librerías utilizadas:

In [3]:
import pandas as pd

import nltk     # for stopwords
import spacy    # for tokenization and lemmatization
import re       # for regular expressions

from nltk.corpus import stopwords

## Análisis del dataset

En principio contamos con 164690 tuplas:

In [4]:
df = pd.read_csv("data/noticias_argentinas_clean_2.csv", delimiter=";")
df

,fecha,titular,fuente,categoria
0,14/11/2018 09:08,Trabajadores del Buenos Aires Design cortan la...,Infobae.com,Nacional
1,13/11/2018 14:14,La boda del gobernador Gerardo Morales: tapas ...,Clarín.com,Nacional
2,14/11/2018 10:08,Cumbre del G20: qué calles estarán cortadas y ...,iprofesional.com,Nacional
3,14/11/2018 02:02,Una fractura que confirma la candidatura de Cr...,LA NACION (Argentina.),Nacional
4,14/11/2018 09:03,Infierno grande: ola de divorcios en un pueblo...,Diario El Día,Nacional
...,...,...,...,...
164685,19/8/2019 09:13,¡Que lo vengan a Berlín!,Olé,Noticias destacadas
164686,20/8/2019 09:56,"Con datos de la NASA, identifican 10 mil millo...",LaRepública.pe,Noticias destacadas
164687,19/8/2019 16:20,3 enfermedades de transmisión sexual que puede...,La 100,Noticias destacadas
164688,19/8/2019 20:12,Hallan evidencia de que la meditación plena ay...,Clarín,Noticias destacadas


Al enumerar las categorías nos encontramos con el valor NaN, indicando que hay tuplas sin especificar su categoría:

In [5]:
df['categoria'].unique()

array(['Nacional', 'Destacadas', 'Deportes', 'Salud',
       'Ciencia y Tecnologia', 'Entretenimiento', 'Economia',
       'Internacional', nan, 'Noticias destacadas'], dtype=object)

Al contar los valores por cada una:

In [6]:
print(df.groupby('categoria')['titular'].count())

categoria
Ciencia y Tecnologia      3856
Deportes                  3855
Destacadas                3859
Economia                  3850
Entretenimiento           3850
Internacional             3850
Nacional                  3860
Noticias destacadas     133819
Salud                     3840
Name: titular, dtype: int64


Además, al observar el dataset podemos ver que existen tuplas que se encuentran repetidas. Teniendo esto en cuenta, contamos nuevamente las noticias:

In [7]:
print(df.groupby('categoria')['titular'].nunique())

categoria
Ciencia y Tecnologia      889
Deportes                 1803
Destacadas               2195
Economia                 1186
Entretenimiento          1506
Internacional            1739
Nacional                 1582
Noticias destacadas     39456
Salud                     797
Name: titular, dtype: int64


La categoría "Noticias destacadas" tiene una cantidad considerablemente mayor de titulares que el resto. Además, es una categoría general que contempla a noticias de todo tipo. En ejecuciones anteriores esto demostró un error muy grande al estimar. Como también afecta en la performance, vamos a desestimar todas las noticias que caigan dentro de esta categoría... por ahora.

In [8]:
df_filtered = pd.DataFrame(df[df['categoria'] != 'Noticias destacadas'])
print(df_filtered.groupby('categoria')['titular'].count())

categoria
Ciencia y Tecnologia    3856
Deportes                3855
Destacadas              3859
Economia                3850
Entretenimiento         3850
Internacional           3850
Nacional                3860
Salud                   3840
Name: titular, dtype: int64


In [9]:
df_filtered.shape

(30871, 4)

## Preprocesamiento de los datos

Vamos a expresar los títulos como un array conformado por sus palabras relevantes lematizadas:

In [10]:
# stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

# tokenization and lemmatization
nlp = spacy.load("es_core_news_md")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\74005\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [11]:
def remove_numbers_and_symbols(input_string):
    pattern = r'[^a-zA-ZáéíóúÁÉÍÓÚñÑüÜ\s]'
    cleaned_string = re.sub(pattern, '', input_string)
    return cleaned_string

In [12]:
def preprocess_text(title_string : str):

  # removing unnecesary symbols and tokenization
  title = nlp(remove_numbers_and_symbols(title_string))

  lemmas = []

  for tok in title:
    word = tok.lemma_.lower()
    if word not in stop_words:
      lemmas.append(word)

  return lemmas

In [13]:
df_filtered['titular'] = df_filtered['titular'].apply(preprocess_text)

## Armado de vocabulario

Para empezar creamos un diccionario con todas las palabras utilizadas en los titulares para cada categoría, evitando repetidos en cada set:

In [14]:
def get_categories(df : pd.DataFrame):
    return df.dropna(subset=['categoria'])['categoria'].unique()

In [15]:
def get_vocab(df : pd.DataFrame, categories: list):

  vocab = {}

  for category in categories:

    # set of words used in the current category
    cat_vocab = set()

    # subset of training data of the current category
    cat_train_df = df[df['categoria'] == category]

    for title in cat_train_df['titular']:
      cat_vocab.update(title)

    vocab[category] = list(cat_vocab)
  
  return vocab

## Cálculo de frecuencias

Iteramos los titulares dentro de cada categoría y calculamos la frecuencia de aparición de las palabras dentro de cada categoría:

In [16]:
K_POSSIBLE_VALUES = 2; # word either appears or not

In [17]:
def get_frequencies(df : pd.DataFrame, categories : list, vocab : dict):

  vocab_freq = {}
  laplace_vocab_freq = {}
  titles_by_cat = {}

  for cat_idx, category in enumerate(categories):

    # all the words used the in current category
    word_list = vocab[categories[cat_idx]]

    # dict with the frequencies of every word in the current category
    words_freq = { key: 0 for key in word_list }
    words_laplace_freq = { key: 0 for key in word_list }

    # training data subset of the current category
    cat_train_df = df[df['categoria'] == category]

    # amount of titles in the current category
    cat_title_count = len(cat_train_df['titular'])
    titles_by_cat[category] = cat_title_count

    for word in word_list:
      for title in cat_train_df['titular']:
        if word in title:
          words_freq[word] += (1 / cat_title_count)
          words_laplace_freq[word] += (1 / (cat_title_count+K_POSSIBLE_VALUES))
      words_laplace_freq[word] += (1 / (cat_title_count+K_POSSIBLE_VALUES))

    vocab_freq[category] = words_freq
    laplace_vocab_freq[category] = words_laplace_freq

  return vocab_freq, laplace_vocab_freq, titles_by_cat

## Cálculo de probabilidades

Primero calculamos la probabilidad de que un titular pertenezca a cierta categoría:

In [18]:
def p_cat(categories : list, titles_by_cat : dict):

  title_count = sum(titles_by_cat.values())
  p_cat = { key: 0 for key in categories }

  for category in categories:
    p_cat[category] = titles_by_cat[category] / title_count

  return p_cat

Con esta información y con las frecuencias guardadas en la sección anterior podemos calcular **P(A|categoria)**, es decir, la probabilidad de ocurrencia de un conjunto de palabras dada cierta categoría:

In [19]:
def p_a_cat(conj_a : list, cat_vocab_freq : dict):
  prob = 1
  for word in conj_a:
    prob *= cat_vocab_freq.get(word, 0)
  return prob

In [20]:
def p_a_cat_laplace(conj_a : list, cat_laplace_vocab_freq : dict, total_titles_cat : int):
  prob = 1
  for word in conj_a:
    freq = cat_laplace_vocab_freq.get(word, 0)
    if freq > 0:
      prob *= freq
    else:
      prob *= (1 / (total_titles_cat+K_POSSIBLE_VALUES))
  return prob

Luego obtenemos **P(A)**, la probabilidad de ocurrencia de un conjunto de palabras:

In [21]:
def p_a(conj_a : list, categories : list, prob_cats : dict, vocab_freq : dict, titles_by_cat : dict, laplace_smoothing : bool = False, laplace_vocab_freq : dict = None):
  prob = 0
  for category in categories:
    prob_a_cat = 0
    if laplace_smoothing:
      prob_a_cat = p_a_cat_laplace(conj_a, laplace_vocab_freq[category], titles_by_cat[category])
    else:
      prob_a_cat = p_a_cat(conj_a, vocab_freq[category])
    prob += (prob_a_cat * prob_cats[category])
  return prob

Finalmente podemos calcular **P(categoria|A)**, la probabilidad de que un titular pertenezca a cierta categoría dado su conjunto de palabras:

In [39]:
def p_cat_a(prob_a_cat : float, prob_cat : float, prob_a : float):
    return prob_a_cat * prob_cat / prob_a

## Naive Bayes

Calculamos las probabilidades P(categoria|A) para cada categoría y nos quedamos con la de mayor valor:

In [23]:
def naive_bayes(conj_a : list, categories : list, vocab_freq : dict, titles_by_cat : dict, prob_cats : dict, laplace_smoothing : bool = False, laplace_vocab_freq : dict = None):

  p_cats_a = {}
  prob_a = p_a(conj_a, categories, prob_cats, vocab_freq, titles_by_cat, laplace_smoothing, laplace_vocab_freq)

  for category in categories:

    prob_a_cat = 0
    if laplace_smoothing:
      prob_a_cat = p_a_cat_laplace(conj_a, laplace_vocab_freq[category], titles_by_cat[category])
    else:
      prob_a_cat = p_a_cat(conj_a, vocab_freq[category])

    p_cats_a[category] = p_cat_a(prob_a_cat, prob_cats[category], prob_a)
  
  return max(p_cats_a, key = lambda k: p_cats_a[k]), p_cats_a

## Probando el clasificador

Utilizamos el método K-Fold para futura cross-validation:

In [24]:
from data_split import k_fold_split

Vamos a crear dos conjuntos, uno de entrenamiento y otro de testeo, por lo que nos queda:

In [25]:
k = 2

test_size = df_filtered.shape[0] / k
(test_size * (k-1), test_size)

(15435.5, 15435.5)

In [26]:
train_df, test_df = k_fold_split(df_filtered, k)

In [27]:
def train(df : pd.DataFrame, possible_labels: list):
    vocab = get_vocab(df, possible_labels)
    vocab_freq, laplace_vocab_freq, titles_by_cat = get_frequencies(df, possible_labels, vocab)
    prob_cats = p_cat(possible_labels, titles_by_cat)
    return {
        "possible_labels": possible_labels,
        "vocab": vocab,
        "vocab_freq": vocab_freq,
        "laplace_vocab_freq": laplace_vocab_freq,
        "titles_by_cat": titles_by_cat,
        "prob_cats": prob_cats
    }

In [28]:
possible_labels = get_categories(df_filtered)
trained_params = train(train_df, possible_labels)
selected_class, p_cats_a = naive_bayes(["messi", "pelota"], possible_labels, trained_params["vocab_freq"], trained_params["titles_by_cat"], trained_params["prob_cats"], True, trained_params["laplace_vocab_freq"])

for category in p_cats_a.keys():
    print(f"P ( cat = '{category}' | A ) = {p_cats_a[category]}")

print(f"\nSelected class: {selected_class}")

P ( cat = 'Nacional' | A ) = 0.06448188303399563
P ( cat = 'Destacadas' | A ) = 0.021706661206412906
P ( cat = 'Deportes' | A ) = 0.8386970195290678
P ( cat = 'Salud' | A ) = 0.011161431950744224
P ( cat = 'Ciencia y Tecnologia' | A ) = 0.010735906875987833
P ( cat = 'Entretenimiento' | A ) = 0.03158928159227289
P ( cat = 'Economia' | A ) = 0.010819519677703041
P ( cat = 'Internacional' | A ) = 0.010808296133815606

Selected class: Deportes


## Error de clasificación

In [30]:
def predict(test_serie : pd.Series, trained_params : dict):
    selected_class, _ = naive_bayes(test_serie['titular'], trained_params["possible_labels"], trained_params["vocab_freq"], trained_params["titles_by_cat"], trained_params["prob_cats"], True, trained_params["laplace_vocab_freq"])
    return selected_class

In [43]:
def predict_with_dict(test_serie : pd.Series, trained_params : dict):
    selected_class, prob_cats_a = naive_bayes(test_serie['titular'], trained_params["possible_labels"], trained_params["vocab_freq"], trained_params["titles_by_cat"], trained_params["prob_cats"], True, trained_params["laplace_vocab_freq"])
    return selected_class, prob_cats_a

Cross-validation:

In [31]:
from error_functions import compute_classification_error
from df_utils        import get_column_value_dict

In [32]:
test_df_label_dict = get_column_value_dict(test_df, "categoria")
error_cases = compute_classification_error(test_df, test_df_label_dict, predict, trained_params)
total_cases = test_df.shape[0]

In [33]:
print(f"Error en {error_cases} de {total_cases} titulares ({round(error_cases/total_cases*100, 2)}%)")

Error en 2829 de 15435 titulares (18.33%)


## División óptima del conjunto de textos

In [34]:
from error_functions import k_fold_cross_validation

In [35]:
k_splits = k_fold_cross_validation(df_filtered, possible_labels, "categoria", train, predict, 5)

In [36]:
i = 1
for k_split in k_splits:
    print(f"Split #{i}")
    train_set = k_split["train"]
    test_set  = k_split["test"]
    print(f"Error training: {train_set['err']}")
    print(f"Error test: {test_set['err']}")
    print("\n------------------------------------------------\n")
    i += 1

Split #1
Error training: 3088
Error test: 1033

------------------------------------------------

Split #2
Error training: 3064
Error test: 998

------------------------------------------------

Split #3
Error training: 3088
Error test: 979

------------------------------------------------

Split #4
Error training: 3037
Error test: 1027

------------------------------------------------

Split #5
Error training: 3047
Error test: 1065

------------------------------------------------



## Confusion matrix

In [41]:
from confusion_matrix import calculate_confusion_matrix, \
                             calculate_roc_confusion_matrices, \
                             calculate_roc_positive_rates
import numpy as np

In [69]:
prediction_probabilities = dict()
predicted = dict()
for sample_idx, sample in test_df.drop("categoria", axis="columns").iterrows():
    predicted[sample_idx], prediction_probabilities[sample_idx] = predict_with_dict(sample, trained_params)

In [70]:
prediction_probabilities

{29719: {'Nacional': 4.0667688593460267e-07,
  'Destacadas': 1.2658328292876774e-07,
  'Deportes': 9.060623460325341e-08,
  'Salud': 0.9999983280438988,
  'Ciencia y Tecnologia': 2.2476904764279788e-07,
  'Entretenimiento': 1.3599081114867669e-08,
  'Economia': 7.477332525606853e-07,
  'Internacional': 6.198831659439155e-08},
 18791: {'Nacional': 0.000134321441741556,
  'Destacadas': 6.531432586571696e-05,
  'Deportes': 0.00021806312846275058,
  'Salud': 1.9533312198389876e-05,
  'Ciencia y Tecnologia': 6.352600854728542e-06,
  'Entretenimiento': 3.4656039776767385e-06,
  'Economia': 0.9995074798858059,
  'Internacional': 4.546970109329007e-05},
 25288: {'Nacional': 0.9750370089884219,
  'Destacadas': 0.004878256279264375,
  'Deportes': 2.867588577491376e-05,
  'Salud': 1.302115993793615e-07,
  'Ciencia y Tecnologia': 8.859268198216251e-08,
  'Entretenimiento': 0.018883230364230553,
  'Economia': 6.401220676470652e-07,
  'Internacional': 0.0011719695559588867},
 22562: {'Nacional': 8.8

In [48]:
# from math import isnan
# test_df_label_dict = {k: test_df_label_dict[k] for k in test_df_label_dict if not isnan(k)}a

In [50]:
possible_labels

array(['Nacional', 'Destacadas', 'Deportes', 'Salud',
       'Ciencia y Tecnologia', 'Entretenimiento', 'Economia',
       'Internacional'], dtype=object)

In [51]:
predicted

{29719: 'Salud',
 18791: 'Economia',
 25288: 'Nacional',
 22562: 'Internacional',
 763: 'Salud',
 17704: 'Nacional',
 148: 'Economia',
 22344: 'Nacional',
 16152: 'Economia',
 4490: 'Internacional',
 29826: 'Economia',
 26275: 'Salud',
 7030: 'Entretenimiento',
 17014: 'Ciencia y Tecnologia',
 18101: 'Destacadas',
 3612: 'Internacional',
 18203: 'Salud',
 3541: 'Nacional',
 8416: 'Internacional',
 26104: 'Deportes',
 26502: 'Deportes',
 15000: 'Nacional',
 4903: 'Destacadas',
 16534: 'Ciencia y Tecnologia',
 22367: 'Ciencia y Tecnologia',
 8138: 'Ciencia y Tecnologia',
 14852: 'Ciencia y Tecnologia',
 27065: 'Deportes',
 10427: 'Destacadas',
 29573: 'Entretenimiento',
 26495: 'Destacadas',
 4764: 'Salud',
 29989: 'Economia',
 27607: 'Nacional',
 1917: 'Economia',
 25287: 'Nacional',
 5459: 'Nacional',
 17184: 'Economia',
 23201: 'Internacional',
 21809: 'Ciencia y Tecnologia',
 29469: 'Deportes',
 27083: 'Ciencia y Tecnologia',
 11028: 'Entretenimiento',
 16605: 'Salud',
 27242: 'Cienc

In [52]:
test_df_label_dict

{29719: 'Salud',
 18791: 'Economia',
 25288: 'Nacional',
 22562: 'Internacional',
 763: 'Salud',
 17704: 'Destacadas',
 148: 'Economia',
 22344: 'Destacadas',
 16152: 'Economia',
 4490: 'Internacional',
 29826: 'Economia',
 26275: 'Salud',
 7030: 'Entretenimiento',
 17014: 'Ciencia y Tecnologia',
 18101: 'Destacadas',
 3612: 'Internacional',
 18203: 'Salud',
 3541: 'Nacional',
 8416: 'Internacional',
 26104: 'Deportes',
 26502: 'Deportes',
 15000: 'Deportes',
 4903: 'Destacadas',
 16534: 'Ciencia y Tecnologia',
 22367: 'Ciencia y Tecnologia',
 8138: 'Ciencia y Tecnologia',
 14852: 'Ciencia y Tecnologia',
 27065: 'Deportes',
 10427: 'Destacadas',
 29573: 'Entretenimiento',
 26495: 'Destacadas',
 4764: 'Salud',
 29989: 'Economia',
 27607: 'Nacional',
 1917: 'Economia',
 25287: 'Nacional',
 5459: 'Nacional',
 17184: 'Economia',
 23201: 'Internacional',
 21809: 'Ciencia y Tecnologia',
 29469: 'Deportes',
 27083: 'Ciencia y Tecnologia',
 11028: 'Entretenimiento',
 16605: 'Salud',
 27242: 'C

In [ ]:
calculate_confusion_matrix(possible_labels, predicted, test_df_label_dict)

## Metrics

In [65]:
from confusion_matrix import calculate_per_label_confusion_matrix, metrics

In [68]:
per_label_conf_mats = calculate_per_label_confusion_matrix(possible_labels, predicted, test_df_label_dict)

KeyError: nan

## ROC curve

In [56]:
label_dict_p_or_f_cats = {}
for category in possible_labels:
    test_df_label_dict_p_or_f = dict()
    for key in test_df_label_dict.keys():
        test_df_label_dict_p_or_f[key] = "P" if test_df_label_dict[key] == category \
                                    else "N"
    label_dict_p_or_f_cats[category] = test_df_label_dict_p_or_f

In [61]:
label_dict_p_or_f_cats['Salud']

{29719: 'P',
 18791: 'N',
 25288: 'N',
 22562: 'N',
 763: 'P',
 17704: 'N',
 148: 'N',
 22344: 'N',
 16152: 'N',
 4490: 'N',
 29826: 'N',
 26275: 'P',
 7030: 'N',
 17014: 'N',
 18101: 'N',
 3612: 'N',
 18203: 'P',
 3541: 'N',
 8416: 'N',
 26104: 'N',
 26502: 'N',
 15000: 'N',
 4903: 'N',
 16534: 'N',
 22367: 'N',
 8138: 'N',
 14852: 'N',
 27065: 'N',
 10427: 'N',
 29573: 'N',
 26495: 'N',
 4764: 'P',
 29989: 'N',
 27607: 'N',
 1917: 'N',
 25287: 'N',
 5459: 'N',
 17184: 'N',
 23201: 'N',
 21809: 'N',
 29469: 'N',
 27083: 'N',
 11028: 'N',
 16605: 'P',
 27242: 'N',
 26291: 'N',
 29735: 'N',
 19336: 'N',
 1400: 'N',
 8939: 'N',
 20046: 'N',
 17873: 'N',
 22222: 'N',
 18615: 'N',
 339: 'N',
 4662: 'N',
 11853: 'N',
 13882: 'N',
 9018: 'P',
 939: 'N',
 19770: 'N',
 7641: 'N',
 19852: 'N',
 17294: 'N',
 1004: 'P',
 5990: 'N',
 29914: 'N',
 10132: 'P',
 15706: 'N',
 20709: 'N',
 30057: 'N',
 20240: 'N',
 5679: 'N',
 22265: 'N',
 19366: 'N',
 15712: 'N',
 3836: 'N',
 12628: 'N',
 2272: 'N',
 

In [ ]:
roc_conf_mats = {}
for category in possible_labels:
    roc_conf_mats[category] = calculate_roc_confusion_matrices(prediction_probabilities, label_dict_p_or_f_cats[category], np.arange(0, 1, 0.0001))

In [ ]:
roc_positive_rates = {}
for category in possible_labels:
    roc_positive_rates[category] = calculate_roc_positive_rates(roc_conf_mats[category])

In [66]:
from plotter import plot_roc_from_positive_rates, plot_multiple_roc_fromm_positive_rates

In [ ]:
plot_multiple_roc_fromm_positive_rates(roc_positive_rates, ["red", "green", "yellow", "blue", "pink", "orange", "cyan", "black"], "Curvas ROC", "roc_curves.png", labels=possible_labels, annotate=True)